# 한빛출판네트워크 - 새로나온책 10 페이지
- https://www.hanbit.co.kr/store/books/new_book_list.html
- 제목, 저자, 출간일, 쪽수, 가격, 역자

In [1]:
import requests, re
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

#### 1. 데이터 가져오기

In [2]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/store/books/new_book_list.html'
url = base_url + sub_url
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

#### 2. 찾으려고 하는 데이터 찾기

In [3]:
lis = soup.select('.sub_book_list')
len(lis)

20

#### 3. 하나의 데이터로 원하는 정보 추출

In [4]:
li = lis[0]
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B2831223974'

- sub page에서 정보를 가져옴

In [5]:
res = requests.get(base_url + href)
book_soup = BeautifulSoup(res.text, 'html.parser')

In [6]:
info = book_soup.select_one('.store_product_info_box')
title = info.find('h3').string
title

'엑셀이 편해지는 파이썬'

In [7]:
book_lis = info.select('.info_list > li')
author = book_lis[0].find('span').get_text().strip()
author

'펠릭스 춤슈타인'

In [8]:
if len(book_lis) == 5:
    translator = '-'
else:
    translator = book_lis[1].find('span').get_text().strip()
translator

'한선용'

In [9]:
date = book_lis[-4].find('span').get_text().strip()
date

'2022-04-20'

In [10]:
page = book_lis[-3].find('span').get_text().strip().split()[0]
page = int(page)
page

388

In [12]:
payments = book_soup.select_one('.payment_box.curr').find_all('p')
sales = payments[1].select_one('.pbr').get_text().split('(')[0]
sales = int(re.sub('[^0-9]','',sales))
sales

25200

#### 4. 한 페이지에 있는 모든 데이터 추출

In [13]:
lines = []
for li in lis:
    book_url = li.find('a')['href']
    res = requests.get(base_url + book_url)
    book_soup = BeautifulSoup(res.text, 'html.parser')

    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').string
    book_lis = info.select('.info_list > li')
    author = book_lis[0].find('span').get_text().strip()
    if len(book_lis) == 5:
        translator = '-'
    else:
        translator = book_lis[1].find('span').get_text().strip()
    date = book_lis[-4].find('span').get_text().strip()
    page = book_lis[-3].find('span').get_text().strip().split()[0]
    
    payments = book_soup.select_one('.payment_box.curr').find_all('p')
    sales = payments[1].select_one('.pbr').get_text().split('(')[0]
    sales = int(re.sub('[^0-9]','',sales))

    lines.append([title,author,translator,date,page,sales])

In [14]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','판매가'])
df.tail()

,제목,저자,번역,출간일,페이지,판매가
15,맛있는 디자인 애프터 이펙트 CC 2022,이수정,-,2022-03-25,528,21600
16,"STEM@CookBook, Beer의 정역학과 재료역학(3판)","Ferdinand P. Beer , E. Russell Johnston, Jr. ,...","김경식 , 박재균 , 오창국 , 이정휘",2022-03-22,772,39000
17,3일 만에 끝내는 초등 글쓰기 트레이닝 북,안도 히데아키,"이정미 , 콩나물쌤 전병규(감수)",2022-03-21,172,11520
18,도메인 주도 개발 시작하기,최범균,-,2022-03-21,356,25200
19,STEIN 푸리에 해석학,"ELIAS M. STEIN , RAMI SHAKARCHI","한빛수학교재연구소 , 이상혁(감수)",2022-03-18,348,38000


#### 5. 모든 페이지의 데이터 가져오기

In [15]:
lines = []
for page in tqdm(range(1,11)):
    url = f'{base_url}{sub_url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')

    for li in lis:
        book_url = li.find('a')['href']
        res = requests.get(base_url + book_url)
        book_soup = BeautifulSoup(res.text, 'html.parser')

        info = book_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        book_lis = info.select('.info_list > li')
        author = book_lis[0].find('span').get_text().strip()
        if len(book_lis) == 5:
            translator = '-'
        else:
            translator = book_lis[1].find('span').get_text().strip()
        date = book_lis[-4].find('span').get_text().strip()
        page = book_lis[-3].find('span').get_text().strip().split()[0]
        
        payments = book_soup.select_one('.payment_box.curr').find_all('p')
        sales = payments[1].select_one('.pbr').get_text().split('(')[0]
        sales = int(re.sub('[^0-9]','',sales))

        lines.append([title,author,translator,date,page,sales])

  0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','판매가'])
df.tail()

,제목,저자,번역,출간일,페이지,판매가
195,"풀스택 서버리스 : 리액트, AWS, 그래프QL을 이용한 최신 애플리케이션 개발",네이더 다빗,김범준,2021-07-05,216,18900
196,한 권으로 배우는 작고 예쁜 꽃자수,장미나(미나랜드),-,2021-07-05,240,16200
197,"IT CookBook, 처음 만나는 회로이론(2판)",방성완,2021-07-02,400 쪽,9791156645474,23000
198,안전필수 시스템 제어 설계,"김국헌 , 김태효 , 이성섭",-,2021-07-02,488,38000
199,데이터 스토리,낸시 두아르테,"권혜정 , 윤영진(감수)",2021-07-01,240,19800


In [17]:
df.to_csv('한빛신규도서.csv', index=False)